In [11]:
print("Hello, World!")

Hello, World!


### 1. Preparing and chunking Data

In [12]:
import json
from pathlib import Path
from langchain_core.documents import Document

# Load the KB JSON
KB_JSON_PATH = r"../cob_kb_singlefile/cob_kb.json"
items = json.loads(Path(KB_JSON_PATH).read_text(encoding="utf-8"))

docs = []
for item in items:
    text = (item.get("text") or "").strip()
    if not text:
        continue

    docs.append(
        Document(
            page_content=text,
            metadata={
                "id": item.get("id"),
                "title": item.get("title"),
                "category": item.get("category"),
                "source_file": item.get("source_file"),
                "paragraph_index": item.get("paragraph_index"),
                "lang": item.get("lang"),
            },
        )
    )

print("Total docs:", len(docs))
print("Sample metadata:", docs[67].metadata)
print("Sample content:", docs[67].page_content[:200])


Total docs: 70
Sample metadata: {'id': 'cob_kb_ar_p33', 'title': 'COB Solution Knowledge Base', 'category': 'limits_policy', 'source_file': 'COB_KB_AR.md', 'paragraph_index': 33, 'lang': 'ar'}
Sample content: إذا لم يجد البوت إجابة في قاعدة المعرفة المتاحة، يجب أن يصرّح بأنه لا يملك معلومات كافية بدل التخمين. يمكنه طرح سؤال توضيحي واحد؛ وإذا استمر الغموض يقوم بالتصعيد.


### 2. Embedding model

In [13]:
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings

load_dotenv()
api_key = os.getenv('GEMINI_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    api_key=api_key,
    output_dimensionality = 1536
    )

In [14]:
# Test
vector = embeddings.embed_query("hello, world!")
vector[:5]

[-0.023955047, 0.011876456, -0.0033613679, -0.0584139, 0.0015592978]

In [15]:
# Test 
len(vector)

1536

### 3. Vector stores

In [18]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="COB_collection",
    embedding_function=embeddings,
    persist_directory="chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [20]:
ids = vector_store.add_documents(documents=docs)

In [21]:
retriever = vector_store.as_retriever()

In [23]:
# Test 
vector_store.similarity_search("ايه الخدمات اللى بتقدمها الشركه؟")

[Document(id='2dd65583-950e-4c72-bb29-37496e87c4e9', metadata={'id': 'cob_kb_ar_p21', 'title': 'COB Solution Knowledge Base', 'category': 'services_operations', 'lang': 'ar', 'source_file': 'COB_KB_AR.md', 'paragraph_index': 21}, page_content='تقدم COB Solution خدمات قابلة للتخصيص وفقًا لاحتياجات كل عيادة. كما تشجع منشورات عامة على ترتيب اجتماع لتحديد نوع الدعم المطلوب وكيفية قياس النجاح.'),
 Document(id='ff9b0ca4-ddcc-4dae-b2ed-9b7ed8f3f07e', metadata={'source_file': 'COB_KB_AR.md', 'lang': 'ar', 'id': 'cob_kb_ar_p01', 'category': 'company_overview', 'paragraph_index': 1, 'title': 'COB Solution Knowledge Base'}, page_content='شركة COB Solution هي جهة استشارات وخدمات في مجال الرعاية الصحية، وتساعد العيادات والمراكز الطبية على تحسين التشغيل وتطوير سير العمل وتقليل العبء الإداري. الهدف هو تمكين مقدمي الخدمة من التركيز على رعاية المرضى مع رفع كفاءة الأداء والنمو.'),
 Document(id='bca07f1c-5a88-4def-9fa7-bd0e6eb75eba', metadata={'category': 'booking_services_demo', 'source_file': 'COB_KB_A

In [24]:
# Test 
vector_store.similarity_search("What is COBsolution? and where it located?")

[Document(id='5418b2ec-5189-4593-acc7-24a7a0f63014', metadata={'lang': 'en', 'source_file': 'COB_KB_EN.md', 'paragraph_index': 2, 'title': 'COB Solution Knowledge Base', 'category': 'company_overview', 'id': 'cob_kb_en_p02'}, page_content='COB Solution is headquartered in New York, NY. A listed address is 575 8th Ave, New York, NY 10018, United States.'),
 Document(id='f8905383-a8af-4c5f-8704-2fb1b7179706', metadata={'title': 'COB Solution Knowledge Base', 'id': 'cob_kb_ar_p02', 'paragraph_index': 2, 'source_file': 'COB_KB_AR.md', 'lang': 'ar', 'category': 'company_overview'}, page_content='المقر الرئيسي لشركة COB Solution في نيويورك (New York, NY). كما يظهر عنوان مُدرج: 575 8th Ave, New York, NY 10018، الولايات المتحدة.'),
 Document(id='5d21d2e9-f28d-4038-b7ca-2ae3ba867d2a', metadata={'title': 'COB Solution Knowledge Base', 'id': 'cob_kb_en_p03', 'lang': 'en', 'source_file': 'COB_KB_EN.md', 'category': 'contact', 'paragraph_index': 3}, page_content='For general inquiries, COB Solution